#### 🔧 Étape 1 : Conversion des documents en texte brut


In [62]:
from utils.convert_to_text import convert_to_text

# Exemple d'utilisation :
cv_text = convert_to_text("test_data/cv.txt")
job_text = convert_to_text("test_data/job.txt")


In [63]:

# Affichage partiel pour vérification
print(cv_text[:500])

Mohamed OUABBI
Full-Stack Web Developer | Digital Transformation
Email: mohamed.ouabbi@example.com | Phone: +212 6 12 34 56 78
LinkedIn: linkedin.com/in/mohamedouabbi | GitHub: github.com/mouabbi
Summary

Passionate full-stack developer with over 2 years of experience in designing, developing, and maintaining modern web applications. Strong expertise in JavaScript, React, Node.js, REST API integration, and managing both relational and NoSQL databases. Interested in cloud solutions, microservices


#### Extraction des compétences grâce à ``SkillNER``

- La fonction extract_skills prend un texte brut (txt) en entrée et retourne une liste de compétences extraites.


In [ ]:
from utils.extract_skills import extract_skills

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [64]:
# Extraire les compétences
cv_skills = extract_skills(cv_text)
job_skills = extract_skills(job_text)

# Affichage des compétences extraites
for skill in job_skills[:10]:
    print(f"Skill: {skill['skill_name']}, Type: {skill['skill_type']}, Match Type: {skill['match_type']}, Score: {skill['score']}")

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\skillNer\utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


Skill: Web Platforms, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Software Development, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Software Development Life Cycle, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: React.js, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Node.js, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Next.js, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Version Control, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Computer Science, Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: JSON Web Token (JWT), Type: Hard Skill, Match Type: full_matches, Score: 1
Skill: Positivity, Type: Soft Skill, Match Type: ngram_scored, Score: 0.5491300225257874


### here is an exemple of ``SkillNER`` usage : 

In [43]:
# imports
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# extract skills from job_description
job_description = """
I am a highly motivated software engineer with strong skills in Python, 
Machine Learning, and Web Development. I thrive in collaborative environments 
thanks to my excellent communication and problem-solving abilities, 
and I continuously seek to improve through critical thinking and teamwork."""

annotations = skill_extractor.annotate(job_description)

skill_extractor.describe(annotations)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


### Calcul du Score de Similarité avec `skill2vec`

In [65]:
# Charger un modèle Word2Vec déjà entraîné
from gensim.models import Word2Vec

model = Word2Vec.load("models/skill2vec_10k_trained.model")

# Vérification
print(model.wv.most_similar("java", topn=5))

[('core java', 0.8455055356025696), ('j2ee', 0.8368200063705444), ('hibernate', 0.8030577898025513), ('maven', 0.7879219651222229), ('webservices', 0.7874660491943359)]


In [113]:
from utils.skill2vec_matching import skillset_similarity

# Extraction, conversion en minuscules et suppression des doublons
cv_skills_ = list(set(skill["skill_name"].lower() for skill in cv_skills))
job_skills_ = list(set(skill["skill_name"].lower() for skill in job_skills))


In [114]:

similarity_score = skillset_similarity(cv_skills_, job_skills_, model)
print(f"Score de similarité entre les compétences du CV et de l'offre : {similarity_score:.4f}")

Score de similarité entre les compétences du CV et de l'offre : 0.9877


#### Et voilà, Skill2Vec  donne un score parfait !

Maintenant, on a créé et organisé cela dans une classe afin de structurer les choses. La classe Skill2VecMatching prend en entrée le CV (sous forme de texte ou de chemin), l'offre d'emploi (texte ou chemin également), ainsi qu'un modèle Word2Vec pré-entraîné, et fournit un score de similarité entre les compétences extraites

In [1]:
from Skill2VecMatching import Skill2VecMatching

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [ ]:
competence_module = Skill2VecMatching("test_data/cv.txt", "test_data/job.txt")

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\skillNer\utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


In [4]:
competence_module.get_similarity_score()

0.98769516